<a href="https://colab.research.google.com/github/Chenhere22/Two-Campaigns-Performance_A-B-testing/blob/main/A_B_testing_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [28]:
control_df=pd.read_csv('/content/control_group.csv', sep=';')
test_df=pd.read_csv('/content/test_group.csv', sep=';')

Find null and fill in with median

In [37]:
test_df.isna().sum()
control_df.isna().sum()

,0
Campaign Name,0
Date,0
Spend [USD],0
# of Impressions,0
Reach,1
# of Website Clicks,1
# of Searches,1
# of View Content,1
# of Add to Cart,1
# of Purchase,0


In [39]:
control_df.fillna(control_df.select_dtypes(include=['number']).median(), inplace=True)


Metrics: Purchase

In [31]:
control_purchases = control_df['# of Purchase']
test_purchases = test_df['# of Purchase']

In [32]:
t_stat_purchases, p_value_purchases = stats.ttest_ind(control_purchases, test_purchases)
print("t-statistic:", t_stat_purchases)
print("p-value:", p_value_purchases)

t-statistic: 0.01638380705469325
p-value: 0.9869844280920268


Metrics: Impression

In [34]:
test_impressions=test_df['# of Impressions']
control_impressions = control_df['# of Impressions']

In [36]:
t_stat_impressions, p_value_impressions = stats.ttest_ind(control_impressions, test_impressions)
print("Impressions t-statistic:", t_stat_impressions)
print("p-value:", p_value_impressions)

Impressions t-statistic: 4.986991949145323
p-value: 5.889238456479816e-06


Metrics: Clicks

In [40]:
test_clicks=test_df['# of Website Clicks']
control_clicks=control_df['# of Website Clicks']
t_stat_clicks, p_value_clicks = stats.ttest_ind(control_clicks, test_clicks)
print("Clicks t-statistic:", t_stat_clicks)
print("p-value:", p_value_clicks)

Clicks t-statistic: -1.611568094917984
p-value: 0.11248505059092864


Metrics: Searches

In [42]:
test_searches=test_df['# of Searches']
control_searches=control_df['# of Searches']
t_stat_searches, p_value_searches = stats.ttest_ind(control_searches, test_searches)
print("Clicks t-statistic:", t_stat_searches)
print("p-value:", p_value_searches)

Clicks t-statistic: -1.1235887075031887
p-value: 0.2658167131206946


Metrics: Add to Cart

In [43]:
test_add_to_cart=test_df['# of Add to Cart']
control_add_to_cart=control_df['# of Add to Cart']
t_stat_cart,p_value_chart=stats.ttest_ind(control_add_to_cart, test_add_to_cart)
print("Add to chart t-statistic:", t_stat_cart)
print("p-value:", p_value_chart)

Add to chart t-statistic: 4.336005165165246
p-value: 5.8568346002878074e-05


# Machine learning

ML 1: Random Forest

In [56]:
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Combine control and test data for machine learning purposes
control_df['Group'] = 0  # Label control group as 0
test_df['Group'] = 1  # Label test group as 1
data = pd.concat([control_df, test_df], axis=0)

# Fill missing values in numeric columns with the median value
data.fillna(data.select_dtypes(include=['number']).median(), inplace=True)

# Feature selection
# Rename columns to remove disallowed characters
data.rename(columns={
    'Spend [USD]': 'Spend_USD',
    '# of Impressions': 'Num_Impressions',
    '# of Website Clicks': 'Num_Website_Clicks',
    '# of Searches': 'Num_Searches',
    '# of View Content': 'Num_View_Content',
    '# of Add to Cart': 'Num_Add_to_Cart'
}, inplace=True)

# Update the features list to match the new column names
features = [
    'Spend_USD', 'Num_Impressions', 'Reach', 'Num_Website_Clicks',
    'Num_Searches', 'Num_View_Content', 'Num_Add_to_Cart'
]
X = data[features]
y = data['Group']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{report}")


Accuracy: 0.8333333333333334
Confusion Matrix:
[[7 1]
 [2 8]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.88      0.82         8
           1       0.89      0.80      0.84        10

    accuracy                           0.83        18
   macro avg       0.83      0.84      0.83        18
weighted avg       0.84      0.83      0.83        18



ML 2: XGBoost

In [57]:
# Train an XGBoost Classifier
xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{report}")

Accuracy: 0.8333333333333334
Confusion Matrix:
[[8 0]
 [3 7]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       1.00      0.70      0.82        10

    accuracy                           0.83        18
   macro avg       0.86      0.85      0.83        18
weighted avg       0.88      0.83      0.83        18



In [ ]:
XGBoost has better performance than Random Forest.